In [1]:
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os
import openai
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
import pandas as pd
from tqdm import tqdm


In [2]:
curated_data = pd.read_csv('../data/benchmark_data/mcq_questions.csv')


In [3]:
LLM_MODEL = 'gpt-4'


In [4]:

load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
API_KEY = os.environ.get('API_KEY')
API_VERSION = os.environ.get('API_VERSION')
RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = API_VERSION
chat_deployment_id = LLM_MODEL
chat_model_id = chat_deployment_id
temperature = 0
chat_model = AzureChatOpenAI(openai_api_key=API_KEY, openai_api_version=API_VERSION, azure_deployment=chat_deployment_id, azure_endpoint=RESOURCE_ENDPOINT, temperature=temperature)
load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
username = os.environ.get('NEO4J_USER')
password = os.environ.get('NEO4J_PSW')
url = os.environ.get('NEO4J_URL')
database = os.environ.get('NEO4J_DB')



/Users/karthiksoman/anaconda3/envs/cypher_rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
graph = Neo4jGraph(url=url, username=username, password=password, database = database)
chain = GraphCypherQAChain.from_llm(chat_model, graph=graph, verbose=True, validate_cypher=True,return_intermediate_steps=True)



In [16]:
%%time

cypher_rag_out = []
for index, row in tqdm(curated_data.iterrows()):
    question = row['text']
    with get_openai_callback() as cb:
        try:
            cypher_rag_answer = chain.run(query=question, return_final_only=True, verbose=False)
        except ValueError as e:
            cypher_rag_answer = None
                
    cypher_rag_out.append((row['text'], row['correct_node'], cypher_rag_answer, cb.total_tokens))    


0it [00:00, ?it/s]



> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]


282it [00:09, 29.53it/s]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...


282it [00:21, 29.53it/s]

...


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Gene)-[:RESPONSE_TO_mGrC]->(c:Compound) WHERE g.variant IN ['rs1126809', 'rs416223', 'rs12722502', 'rs9419958', 'rs1333049'] AND c.name IN ['age-related hearing impairment', 'skin sensitivity to sun'] RETURN g.variant, c.name;
Full Context:
[]


305it [03:18,  7.47s/it]


> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Gene)-[:RESPONSE_TO_mGrC]->(c:Compound) WHERE g.variant IN ['rs2447827', 'rs1937455', 'rs62324212', 'rs12205199', 'rs4482879'] AND c.disease IN ['atopic asthma', 'autoimmune disease'] RETURN g.variant, c.disease
Full Context:
[]


306it [03:26,  1.48it/s]


> Finished chain.
CPU times: user 1.83 s, sys: 194 ms, total: 2.03 s
Wall time: 3min 26s


In [18]:
cypher_rag_out_df = pd.DataFrame(cypher_rag_out, columns=['text', 'label', 'cypher_rag_answer', 'total_tokens'])


In [19]:

save_path = '../data/results'
os.makedirs(save_path, exist_ok=True)
cypher_rag_out_df.to_csv(os.path.join(save_path, 'cypher_rag_mcq_output.csv'), index=False)

